In [91]:
import json
import inspect
from openai import OpenAI
import os

# Got from https://github.com/john-carroll-sw/chat-completions-function-calling-examples/blob/master/func_get_weather.py


def check_args(function, args):
    """
    Check if the correct arguments are provided to a function.
    - Uses the inspect module to get the function signature
    - Compares the function signature with the provided arguments

    Args:
        function (callable): The function to check the arguments for.
        args (list): The arguments provided to the function.

    Returns:
        bool: True if the correct arguments are provided, False otherwise.

    """
    sig = inspect.signature(function)
    params = sig.parameters

    # Check if there are extra arguments
    for name in args:
        if name not in params:
            return False
    # Check if the required arguments are provided
    for name, param in params.items():
        if param.default is param.empty and name not in args:
            return False

    return True




def get_function_and_args(tool_call, available_functions):
    """
    Retrieves the function and its arguments based on the tool call.
    Verifies if the function exists and has the correct number of arguments.

    Args:
        tool_call (ToolCall): The tool call object containing the function name and arguments.
        available_functions (dict): A dictionary of available functions.

    Returns:
        tuple: A tuple containing the function to call and its arguments.
            If the function or arguments are invalid, returns an error message and None.
    """
    # verify function exists
    if tool_call.function.name not in available_functions:
        return "Function " + tool_call.function.name + " does not exist", None
    function_to_call = available_functions[tool_call.function.name]

    # verify function has correct number of arguments
    function_args = json.loads(tool_call.function.arguments)
    if check_args(function_to_call, function_args) is False:
        return "Invalid number of arguments for function: " + tool_call.function.name, None

    return function_to_call, function_args






client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
DEPLOYMENT_NAME = 'gpt-4o'

# Example function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps(
            {"location": "San Francisco", "temperature": "72", "unit": unit}
        )
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})


def run_conversation():
    # Step 1: send the conversation and available functions to the model
    messages = [
        { 
            "role": "system", 
            "content": """
                You are a helpful assistant.
                You have access to a function that can get the current weather in a given location.
                Determine a reasonable Unit of Measurement (Celsius or Fahrenheit) for the temperature based on the location.
            """
        },
        {
            "role": "user",
            "content": "What's the weather like in San Francisco, Tokyo, and Paris?",
        }
    ]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": """
                    Get the current weather in a given location. 
                    Note: any US cities have temperatures in Fahrenheit
                """,
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {
                            "type": "string", 
                            "description": "Unit of Measurement (Celsius or Fahrenheit) for the temperature based on the location",
                            "enum": ["celsius", "fahrenheit"]
                        },
                    },
                    "required": ["location"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model=DEPLOYMENT_NAME,
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
        temperature=0,  # Adjust the variance by changing the temperature value (default is 0.8)
    )

    response_message = response.choices[0].message
    print(response_message)
    tool_calls = response_message.tool_calls
    print(tool_calls)

    # Step 2: check if the model wanted to call a function
    if tool_calls:

        messages.append(response_message)  # extend conversation with assistant's reply
        print(messages)
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        
        for tool_call in tool_calls:

            # Step 3: call the function
            # Note: the JSON response may not always be valid; be sure to handle errors
            function_name = tool_call.function.name

            # get the function and arguments
            function_to_call, function_args = get_function_and_args(tool_call, available_functions)
            
            # call the function
            function_response = function_to_call(**function_args)

            # Step 4: send the info for each function call and function response to the model
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response

        second_response = client.chat.completions.create(
            model=DEPLOYMENT_NAME,
            messages=messages,
            temperature=0,  # Adjust the variance by changing the temperature value (default is 0.8)
        )  # get a new response from the model where it can see the function response
        return second_response
    

result = run_conversation()

message_content = result.choices[0].message.content
print(message_content)


ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_4fEmB5vMDFlNtaUtDen88QsQ', function=Function(arguments='{"location": "San Francisco, CA", "unit": "fahrenheit"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_uAy62r4yVQPG8AUpWJ2xNYxC', function=Function(arguments='{"location": "Tokyo, Japan", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_8bsRgn6o9Y2dM4d7dLFyzOBk', function=Function(arguments='{"location": "Paris, France", "unit": "celsius"}', name='get_current_weather'), type='function')])
[ChatCompletionMessageToolCall(id='call_4fEmB5vMDFlNtaUtDen88QsQ', function=Function(arguments='{"location": "San Francisco, CA", "unit": "fahrenheit"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_uAy62r4yVQPG8AUpWJ2xNYxC', function=Function(arguments='{"loca

In [87]:
messages

[{'role': 'user', 'content': 'What is the square of 5?'},
 {'role': 'tool_calls',
  'content': 'Function(arguments=\'{"number":5}\', name=\'square_number\')'},
 {'role': 'tool',
  'content': 'The output from using the square_number function is 25.'}]